In [1]:
import time
import pandas as pd
import requests
import json
from typing import List

In [2]:
# %pip install polars
import polars as pl

In [3]:
from extract_data import get_URLs,\
get_incident_ids, generate_canadian_avalanche_data

Time to taken for extract_data.py to run: 0.0s.


In [4]:
url = "https://incidents.avalanche.ca/public/incidents/?format=json"

In [5]:
URLS = get_URLs(url)

In [6]:
ids = get_incident_ids(URLS)

In [ ]:
def generate_canadian_avalanche_data(incident_ids: List[str]) -> str:
    df = pd.DataFrame()
    for id in incident_ids:
        data = requests.get(
            "https://incidents.avalanche.ca/public/incidents/{}/?format=json".format(id)
        ).json()
        new_df = pd.json_normalize(data)
        df = pd.concat([df, new_df])
    df.to_csv("./data/can_avs_raw.csv")

    return "Data was saved."

In [13]:
ids[0]

'6dad9a7f-e732-43cf-8058-5fcac24c7041'

In [14]:

src = "https://incidents.avalanche.ca/public/incidents/6dad9a7f-e732-43cf-8058-5fcac24c7041/?format=json"
pd.read_json(src)

ValueError: arrays must all be same length

In [15]:
data = requests.get(src).json()
pd.json_normalize(data)

,id,ob_date,location,location_desc,location_coords,location_coords_type,location_elevation,location_province,num_involved,num_injured,...,weather_obs.temp_min,weather_obs.temp_trend,weather_obs.wind_speed,weather_obs.wind_dir,weather_obs.sky,weather_obs.precip,snowpack_obs.hs,snowpack_obs.hn24,snowpack_obs.hst,snowpack_obs.hst_reset
0,6dad9a7f-e732-43cf-8058-5fcac24c7041,2023-04-22,Lake Louise West Bowl,Lake Louise Ski Resort,"[51.467515, -116.145756]",Lat/lng,2550.0,AB,2,1,...,None,None,None,None,None,None,None,None,None,None


In [27]:
grades_with_list = {key:[value] for key, value in data.items()}
pl.DataFrame(data).explode("avalanche_obs").unnest("avalanche_obs")

ShapeError: could not create a new dataframe: series "id" has length 8 while series "location_coords" has length 2